In [ ]:
import cv2

In [ ]:
from src.imageoverlay.classification_decorator import ClassificationDecorator as ClassDecorator
from src.imageoverlay.framecomponent import CachingLandmarkComponent as LandmarkComponent
from src.imageoverlay.multifacesloopdecorator import MultiFacesLoopDecorator as AllFaces
from src.imageoverlay.framecomponent import LandmarkOverlayDecorator
from src.machinelearningsuite.landmarkdetector import LandmarkDetector
from src.machinelearningsuite.predictorinterface import PredictorInterface
from src.imageoverlay.configured_overlay_decorator import ConfiguredOverlayDecorator as SpriteDecorator

## Method to display streams in notebook 

In [ ]:
import IPython
import PIL.Image
from io import StringIO
from io import BytesIO
from IPython.display import clear_output
import cv2
from time import sleep

In [ ]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, fmt='jpeg'):
    f = BytesIO()
    something = PIL.Image.fromarray(a)
    something.save(f, fmt)
    clear_output(wait=True)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

## Video parameters

In [ ]:
scale = 0.6

In [ ]:
def construct_stream():
    # return cv2.VideoCapture("/media/local/videos/obama.mp4")
    return cv2.VideoCapture(0)

## Main loop method

In [ ]:
def run_pipeline_with_end_component(component):
    try:
        video_stream = construct_stream()
        while True:
            _, skipped_frame = video_stream.read()
            ret, frame = video_stream.read()
            if frame is None:
                continue
            frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
            base_component.set_image(frame)
            output = component.get_image()
            output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
            showarray(output)
    except KeyboardInterrupt:
        video_stream.release()
        print("Stream stopped")
    video_stream.release()

## Decorator pipeline

In [ ]:
# Instanciate a new landmark detector
detector_data_path = '/media/local/data/shape_predictor_68_face_landmarks.dat'
landmark_detector = LandmarkDetector(predictor_file=detector_data_path)

# Create a frame component with landmarks
base_component = LandmarkComponent(landmark_detector)

In [ ]:
landmarks = LandmarkOverlayDecorator(base_component)

In [ ]:
# Instantiate and initialize the trained predictor
predictor = PredictorInterface('./examples/mouth.pkl')
predictor.initialize()

In [ ]:
# Add decorator for the predictor
predictor_decorator = ClassDecorator(parent_component=landmarks, classifier=predictor)
hat = SpriteDecorator(base_file_name='./sprites/Party_Hat')
predictor_decorator.set_decorator_for_class(hat, 0)

In [ ]:
multifaces = AllFaces.wrap(predictor_decorator)

## Run

In [ ]:
run_pipeline_with_end_component(multifaces)

## Useful bash commands

(Change the cell types from 'raw' to 'code' by pressing the 'y' key, then execute)

Check the available video devices

Clean up the output in the notebook

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace --clear-output Barco_Facial_Landmarks.ipynb

Compress the code for downloading